# Imports

In [ ]:
from keras.layers import LSTM, Input, Embedding, Dense, TimeDistributed
from keras.models import Model
import pickle
import numpy as np
from datetime import datetime as dt

# Funções Auxiliares

In [ ]:
def load_obj(name):
    with open( name, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def montar_sentenca(indices_sentenca):
    indices = np.zeros((43), dtype=np.int16)
    i = 0
    while i < len(indices_sentenca):
        indices[i] = indices_sentenca[i]
        i += 1
    return indices

def montar_manchete(indices_manchetes):
    indices_entrada = np.zeros((20), dtype=np.int16)
    saidas = np.zeros((20, len(ind2token_manchetes)), dtype=np.int16)
    i = 0
    while i < len(indices_manchetes) - 1 and i < 20:
        indices_entrada[i] = indices_manchetes[i]
        saidas[i][indices_manchetes[i+1]] = 1
        i += 1
    return indices_entrada, saidas

def montar_bloco(exemplos_batch):
    sentencas_indices = []
    manchetes_indices = []
    saidas = []
    for exemplo in exemplos_batch:
        sentencas_indices.append(montar_sentenca(exemplo['indices_sentenca']))
        indices_manchete, saida = montar_manchete(exemplo['indices_manchete'])
        manchetes_indices.append(indices_manchete)
        saidas.append(saida)
    sentencas_indices = np.array(sentencas_indices, dtype=np.int16)
    manchetes_indices = np.array(manchetes_indices, dtype=np.int16)
    saidas = np.array(saidas)
    return sentencas_indices, manchetes_indices, saidas

# Arquivos Necessários

In [ ]:
ind2token_manchetes = load_obj("ind2token_manchetes.pkl")
token2ind_manchetes = load_obj("token2ind_manchetes.pkl")
ind2token_sentencas = load_obj("ind2token_sentencas.pkl")
token2ind_sentencas = load_obj("token2ind_sentencas.pkl")
ind2tag = load_obj("ind2tag.pkl")
tag2ind = load_obj("tag2ind.pkl")
representacoes_manchetes_100 = np.load("representacoes_manchetes_100.npy")
representacoes_sentencas_200 = np.load("representacoes_sentencas_200.npy")

# Configurações do Modelo

In [ ]:
M = 43
N = 20
tamanho_embedding_sentenca = 200
tamanho_embedding_manchete = 100
E = representacoes_manchetes_100.copy()
F = representacoes_sentencas_200.copy()

# O modelo encoder-decoder

In [ ]:
sentenca_entrada = Input(shape=(M,))
manchete_entrada = Input(shape=(N,))
embeddings_sentenca = Embedding(len(ind2token_sentencas), tamanho_embedding_sentenca, weights=[F],
                                trainable=True, mask_zero=True)(sentenca_entrada)
embeddings_manchete = Embedding(len(ind2token_manchetes), tamanho_embedding_manchete, weights=[E], 
                               trainable=True, mask_zero=True)(manchete_entrada)

# Codificador
saidas_encoder, encoder_h, encoder_c = LSTM(200, return_state=True)(embeddings_sentenca)

# Decodificador
sequencia_gerada, _, _ = LSTM(200, return_sequences=True, return_state=True)(embeddings_manchete, 
                                                                             initial_state=[encoder_h, encoder_c])

saidas = TimeDistributed(Dense(len(ind2token_manchetes), activation='softmax'))(sequencia_gerada)

In [ ]:
model = Model([sentenca_entrada, manchete_entrada], saidas)
model.compile('rmsprop', loss="categorical_crossentropy")
model.summary()

# Configurações do treinamento

In [ ]:
num_epochs = 30
batch_size = 16
tamanhos_treinamento = list(range(23,44))
losses_treinamento = []
losses_validacao = []

# Treinamento

In [ ]:
inicio = dt.now()
for epoch in range(num_epochs):
    # Embaralha os tamanhos
    np.random.shuffle(tamanhos_treinamento)
    # Carrega cada bloco e treina
    losses_treinamento_epoch = []
    print("Início do treino epoch ", str(epoch))
    for tamanho in tamanhos_treinamento:
        exemplos = load_obj("exemplos_treinamento_" + str(tamanho) + ".pkl")
        sentencas, manchetes, saidas = montar_bloco(exemplos)
        history = model.fit(x=[sentencas, manchetes], y=saidas, batch_size=batch_size, verbose=1)
        losses_treinamento_epoch.append(history.history['loss'])
        
    loss_treinamento = np.mean(losses_treinamento_epoch)
    losses_treinamento.append(loss_treinamento)
    # Validação
    losses_validacao_epoch = []
    print("Início da validação epoch ", str(epoch))
    for tamanho in tamanhos_treinamento:
        exemplos_validacao = load_obj("exemplos_validacao_" + str(tamanho) + ".pkl")
        sentencas, manchetes, saidas = montar_bloco(exemplos_validacao)
        losses_validacao_epoch.append(model.evaluate(x=[sentencas, manchetes], y=saidas, verbose=1))
    loss_validacao = np.mean(losses_validacao_epoch)
    losses_validacao.append(loss_validacao)
    print("Epoch ", str(epoch + 1), ". Loss treinamento: ", str(loss_treinamento), ". Loss validação: ", 
          str(loss_validacao), "\nTempo total: ", str(dt.now() - inicio))
    model.save("salvar_baseline/model_baseline_com_embeddings_" + str(epoch) + ".h5")
    model.save_weights("salvar_baseline/model_baseline_com_embeddings_so_pesos_" + str(epoch) + ".h5")
    save_obj(losses_validacao, "salvar_baseline/losses_validacao_" + str(epoch) + ".pkl")
    save_obj(losses_treinamento, "salvar_baseline/losses_treinamento_" + str(epoch) + ".pkl")